In [1]:
from ophyd_async.core import DeviceCollector, StaticPathProvider, AutoIncrementFilenameProvider
from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky.plans import count, scan 
# from ophyd.sim import noisy_det as det
from databroker import Broker
import asyncio
#from epics import caget, caput,cainfo
from enum import Enum

from p99_bluesky.devices.p99 import FilterMotor, SampleAngleStage, p99StageSelections, ThreeAxisStage, Andor2Ad
from p99_bluesky.plans.fast_scan import fast_scan_1d, fast_scan_grid
from p99_bluesky.plans.stxm import stxm_fast,get_velocity_and_step_size, stxm_step
loop = asyncio.get_event_loop()
bec = BestEffortCallback()
RE = RunEngine({})
db = Broker.named('temp')
RE.subscribe(db.insert)

0

In [2]:
andor_data_patt_provider = StaticPathProvider(filename_provider= AutoIncrementFilenameProvider(base_filename="andor2"),
                                              directory_path="/dls/p99/data/2024/cm37284-2/processing/writenData")

In [3]:
with DeviceCollector():
        xyz = ThreeAxisStage(prefix="BL99P-MO-STAGE-02:", name="xyz")
        det_andor = Andor2Ad(prefix="BL99P-EA-DET-03:", name= "det_andor",path_provider=andor_data_patt_provider)


In [12]:
RE(count([det_andor],2))

Run aborted
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/asyncio/tasks.py", line 500, in wait_for
    return fut.result()
           ^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/asyncio/queues.py", line 158, in get
    await getter
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/venv/lib/python3.11/site-packages/ophyd_async/core/_detector.py", line 262, in trigger
    async for index in self.writer.observe_indices_written(
  File "/venv/lib/python3.11/site-packages/ophyd_async/epics/adcore/_hdf_writer.py", line 139, in observe_indices_written
    async for num_captured in observe_value(self.hdf.num_captured, timeout):
  File "/venv/lib/python3.11/site-packages/ophyd_async/core/_signal.py", line 480, in observe_value
    item = await get_value()
           ^^^^^^^^^^^^^^^^^
  File "/venv/lib/python3.11/site-packages/ophyd_async/core/_signal.py", line 472, in get

FailedStatus: <AsyncStatus, task: <coroutine object StandardDetector.trigger at 0x7f2a7eb77cd0>, errored: TimeoutError()>

CA.Client.Exception...............................................
    Context: "bl99p-ea-serv-01.diamond.ac.uk:41480"
    Source File: modules/ca/src/client/cac.cpp line 1237
    Current Time: Thu Sep 12 2024 15:53:58.062768421
..................................................................


In [ ]:
from ophyd_async.core import set_and_wait_for_value

status = await set_and_wait_for_value(det_andor.hdf.capture, True)
## await det_andor.hdf.capture.read()
await status
await det_andor.hdf.capture.set(False,wait=False)
await det_andor.hdf.capture.read()
status = await set_and_wait_for_value(det_andor.hdf.capture, False)
await status
print("Done")

In [5]:
from p99_bluesky.plans import tiggerImg,takeImg
from collections import defaultdict
from ophyd_async.core import DetectorTrigger

In [9]:

RE(takeImg(det= det_andor,exposure = 0.1,
        n_img = 2,))

('7e0886d5-3485-4cb4-b98a-92f0ec82f84a',)

In [ ]:
from epics import caget,caput

In [ ]:
caget("BL99P-EA-DET-03:HDF5:Capture")

In [7]:
caput("BL99P-EA-DET-03:HDF5:Capture",0)

NameError: name 'caput' is not defined

In [11]:
RE(tiggerImg(dets= det_andor,value = 0.1))

Run aborted
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/asyncio/tasks.py", line 500, in wait_for
    return fut.result()
           ^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/asyncio/queues.py", line 158, in get
    await getter
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/venv/lib/python3.11/site-packages/ophyd_async/core/_detector.py", line 262, in trigger
    async for index in self.writer.observe_indices_written(
  File "/venv/lib/python3.11/site-packages/ophyd_async/epics/adcore/_hdf_writer.py", line 139, in observe_indices_written
    async for num_captured in observe_value(self.hdf.num_captured, timeout):
  File "/venv/lib/python3.11/site-packages/ophyd_async/core/_signal.py", line 480, in observe_value
    item = await get_value()
           ^^^^^^^^^^^^^^^^^
  File "/venv/lib/python3.11/site-packages/ophyd_async/core/_signal.py", line 472, in get

FailedStatus: <AsyncStatus, task: <coroutine object StandardDetector.trigger at 0x7f2a7d20b890>, errored: TimeoutError()>